#**LOAD BTC DATASET**

In [394]:
#from google.colab import files
#uploaded = files.upload()

In [395]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('BTC-USD.csv',index_col = 0,parse_dates = True, na_values = 0).dropna()

data = data[:data.shape[0]-60] # To cut the data and try earlier test timeframes


In [396]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-04-10,7303.815430,7303.815430,6802.475098,6865.493164,6865.493164,4.362284e+10
2020-04-11,6867.440430,6926.069824,6789.920898,6859.083008,6859.083008,3.122209e+10
2020-04-12,6858.067871,7119.947266,6811.078125,6971.091797,6971.091797,3.575957e+10
2020-04-13,6965.616699,6965.616699,6668.259766,6845.037598,6845.037598,3.861931e+10
2020-04-14,6843.281738,6958.557129,6793.821289,6842.427734,6842.427734,3.411043e+10


#**DATA SCALING**



In [397]:
max_=data[['Open','High','Low','Close']].max().max()
min_=data[['Open','High','Low','Close']].min().min()

scl=MinMaxScaler()

X1=(data[['Open','High','Low','Close']]-min_)/(max_-min_)
X2=scl.fit_transform(data[['Volume']].values.reshape(-1,1))
X1=np.array(X1)

data=data.assign(Open=X1[:,0])
data=data.assign(High=X1[:,1])
data=data.assign(Low=X1[:,2])
data=data.assign(Close=X1[:,3])
data=data.assign(Volume=X2[:,0])
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-04-10,0.358093,0.358093,0.332922,0.336086,6865.493164,0.588219
2020-04-11,0.336183,0.339127,0.332291,0.335764,6859.083008,0.420982
2020-04-12,0.335713,0.348861,0.333354,0.341387,6971.091797,0.482175
2020-04-13,0.341113,0.341113,0.326183,0.335059,6845.037598,0.520741
2020-04-14,0.334971,0.340758,0.332487,0.334928,6842.427734,0.459934


In [398]:
data.shape

(2037, 6)

#**DEFINE INPUTS AND TARGET FEATURES**

In [399]:
X=data[['Open','High','Low','Close','Volume']]
y=data.Close.shift(-1)
X.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2020-04-10,0.358093,0.358093,0.332922,0.336086,0.588219
2020-04-11,0.336183,0.339127,0.332291,0.335764,0.420982
2020-04-12,0.335713,0.348861,0.333354,0.341387,0.482175
2020-04-13,0.341113,0.341113,0.326183,0.335059,0.520741
2020-04-14,0.334971,0.340758,0.332487,0.334928,0.459934


In [400]:
timestep=10
X_list=[]
y_list=[]
for i in range(timestep,len(X)):
    X_list.append(np.array(X.iloc[i-timestep:i]))
    y_list.append(y.iloc[i])

test_size=60
X_train=np.array(X_list)[:-test_size]
y_train=np.array(y_list)[:-test_size]
X_test=np.array(X_list)[-test_size:]
y_test=np.array(y_list)[-test_size:]

X_train.shape

(1967, 10, 5)

#**CREATE RNN MODEL**

In [401]:
from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

dropout_ratio=0.5
model=Sequential()
model.add(SimpleRNN(timestep,input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(32))
model.add(Dropout(dropout_ratio))
model.add(Dense(64))
model.add(Dropout(dropout_ratio))
model.add(Dense(128))
model.add(Dropout(dropout_ratio))
model.add(Dense(256))
model.add(Dropout(dropout_ratio))
model.add(Dense(512))
model.add(Dropout(dropout_ratio))
model.add(Dense(1024))
model.add(Dropout(dropout_ratio))
model.add(Dense(2048))
model.add(Dropout(dropout_ratio))
model.add(Dense(1))
model.compile(loss='mean_squared_error', 
              optimizer='adam')
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_18 (SimpleRNN)    (None, 10)                160       
_________________________________________________________________
dense_144 (Dense)            (None, 32)                352       
_________________________________________________________________
dropout_126 (Dropout)        (None, 32)                0         
_________________________________________________________________
dense_145 (Dense)            (None, 64)                2112      
_________________________________________________________________
dropout_127 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_146 (Dense)            (None, 128)               8320      
_________________________________________________________________
dropout_128 (Dropout)        (None, 128)             

#**FIT MODEL AND SAVE BEST WEIGHTS**

In [ ]:
filepath="weights-btc.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
history = model.fit(X_train, y_train, epochs=200, batch_size=50, verbose=1, validation_split=0.2, callbacks=[checkpoint])

Epoch 1/200
31/32 [============================>.] - ETA: 0s - loss: 3.6294
Epoch 00001: val_loss improved from inf to 1.98980, saving model to weights-btc.hdf5
32/32 [==============================] - 2s 65ms/step - loss: 3.6137 - val_loss: 1.9898
Epoch 2/200
31/32 [============================>.] - ETA: 0s - loss: 0.8551
Epoch 00002: val_loss improved from 1.98980 to 0.05363, saving model to weights-btc.hdf5
32/32 [==============================] - 2s 62ms/step - loss: 0.8463 - val_loss: 0.0536
Epoch 3/200
31/32 [============================>.] - ETA: 0s - loss: 0.2549
Epoch 00003: val_loss did not improve from 0.05363
32/32 [==============================] - 2s 60ms/step - loss: 0.2532 - val_loss: 0.0660
Epoch 4/200
31/32 [============================>.] - ETA: 0s - loss: 0.1142
Epoch 00004: val_loss improved from 0.05363 to 0.00403, saving model to weights-btc.hdf5
32/32 [==============================] - 2s 61ms/step - loss: 0.1142 - val_loss: 0.0040
Epoch 5/200
31/32 [===========

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,7))
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["train_loss", "val_loss"])

#**TEST PREDICTIONS**

In [ ]:
model.load_weights(filepath)

descale = (max_-min_)+min_

predict_close=[]
predict_close=np.reshape(model.predict(X_test) * descale,(1,-1))
predict_close=predict_close[0]
performance=pd.DataFrame([predict_close, y_test * descale])
performance=performance.T
performance.columns=['Predicted','Actual']
performance.tail()

In [ ]:
Spread = []
Spread = performance.Actual - performance.Predicted

plt.figure(figsize=(15,7))
plt.plot(performance.Predicted,c='y')
plt.plot(performance.Actual,c='b')
plt.legend(['Predicted_Close','Actual_Close'])
plt.show()

#**MEASURE MODEL PERFORMANCE**

In [ ]:
# TEST FOR PROFIT

# TODO: This part of the cold is recycled from old code, so naming conventions must be matched

# If next predicted price is higher than actual price, buy.
# Otherwise, stay out of the market.

current_position = 0

sample = performance

for idx in range(0, 59):
  if sample.loc[idx, 'Actual'] < sample.loc[idx+1, 'Predicted']:
    current_position = 1
  else:
    current_position = 0
  sample.loc[idx,'Position'] = current_position
  sample.loc[idx, 'Profit'] = current_position*(sample.loc[idx+1, 'Actual'] - sample.loc[idx, 'Actual'])
  
#sample.profit.plot()
#plt.axhline(y=0, color='red')

plt.title("Profit per day over history",fontsize=20)
sample['Wealth'] = sample['Profit'].cumsum()
sample.Wealth.plot()
plt.axhline(y=0, color='red')
plt.title("Accumulated profit in {:.0f} days: {:.2f} ({:.2f}%)".format(
    len(sample.index),
    sample.loc[sample.index[-3], 'Wealth'],
    sample.loc[sample.index[-3], 'Wealth']/sample.loc[sample.index[0], 'Actual']*100),
    fontsize=20)

In [ ]:
print("Market profit: {:.2f}%".format((sample.loc[sample.index[-3], 'Actual']/sample.loc[0, 'Actual']-1)*100))

In [ ]:
# Visualize Model vs Buy&Hold side by side

sample['TotalWealth'] = sample['Wealth'] + sample.Actual[0]
sample.TotalWealth.plot(color="blue")
plt.axhline(y=sample.Actual[0], color='red')
sample.Actual.plot(color='yellow')